<a href="https://colab.research.google.com/github/PurvaChiniya/term_project-/blob/main/Grid_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grid Board Q-Learning

![picture](https://drive.google.com/uc?id=1rs4-aNz9e-wVc-vZn2QGWIojkfXjZmvN)

####The rule is simple. Your agent/robot starts at the left-bottom corner(the ‘start’ sign) and ends at either +1 or -1 which is the corresponding reward. At each step, the agent has 4 possible actions including up, down, left and right, whereas the black block is a wall where your agent won’t be able to penetrate through. In order to make it more straight forward, our first implementation assumes that each action is deterministic, that is, the agent will go where it intends to go. For instance, when the agent decides to take action up at (2, 0), it will land in (1, 0) rather than (2, 1) or elsewhere. (We will add uncertainty in out second implementation) However, it the agents hit the wall, it will remain at the same position.

##1 Deterministic Q Learning

#### Besides Q-function, we are going to add more fun to our game:

*   The agent action is non-deterministic
*   Reward decays with ratio γ

In [98]:
init_reward = np.array([[0.02, 0.03, 0.04, 0], [0.01, 0, 0.02, 0], [0, 0.01, 0.02, 0]])

In [99]:
BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3)
LOSE_STATE = (1, 3)
START = (2, 0)
DETERMINISTIC = True





*   Give Reward: And as a grid game, it needs a State to justify each state(position) of our agent, giving reward according to its state.


*   NxtPosition: When our agent takes an action, the State should have a function to accept an action and return a legal position of next state.




In [100]:
class State:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC
        
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0
    
    def isEndFunc(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True
    
    def nxtPosition(self, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position
        """
        if self.determine:
            if action == "up":
                nxtState = (self.state[0]-1, self.state[1])
            elif action == "down":
                nxtState = (self.state[0]+1, self.state[1])
            elif action == "left":
                nxtState = (self.state[0], self.state[1]-1)
            else:
                nxtState = (self.state[0], self.state[1]+1)
            # if next state legal
            if (nxtState[0] >= 0) and (nxtState[0] <= 2):
                if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                    if nxtState != (1, 1):
                        return nxtState
            return self.state
    
    def showBoard(self):
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')

In [101]:

s = State()
s.state

(2, 0)

#### Agent
our agent should be able to learn from the process and thinks like a human. The key of the magic is value iteration.

#### Value Iteration
What our agent will finally learn is a policy, and a policy is a mapping from state to action, simply instructs what the agent should do at each state. In our case, instead of learning a mapping from state to action, we will leverage value iteration to firstly learn a mapping of state to value(which is the estimated reward) and based on the estimation, at each state, our agent will choose the best action that gives the highest estimated reward.

At first, our gent knows nothing about the grid world(environment), so it would simply initialises all reward as 0. Then, it starts to explore the world by randomly walking around, surely it will endure lots of failure at the beginning, but that is totally fine. Once it reaches end of the game, either reward +1 or reward -1, the whole game reset and the reward propagates in a backward fashion and eventually the estimated value of all states along the way will be updated based on the formula above.

####**The V(St) on the left is the updated value of that state, and the right one is the current non-updated value and α is learning rate.**


![picture](https://drive.google.com/uc?id=1dTb49_GFwMIX72d8YFQBQ-4T7PWPf-GF)


In [102]:
class Agent:
    
    def __init__(self):
        self.states = []
        self.actions = ["up", "down", "left", "right"]
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2
        self.exp_rate = 0.3
        
        # initial state reward
        self.state_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.state_values[(i, j)] = 0  # init_reward[i, j]
    
    def chooseAction(self):
        # choose action with most expected value
        mx_nxt_reward = 0
        action = ""
        
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            for a in self.actions:
                # if the action is deterministic
                nxt_reward = self.state_values[self.State.nxtPosition(a)]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
            # print("current pos: {}, greedy aciton: {}".format(self.State.state, action))
        return action
    
    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        return State(state=position)     
    
    def reset(self):
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd
    
    def play(self, rounds=10):
        i = 0
        while i < rounds:
            # to the end of game back propagate reward
            if self.State.isEnd:
                # back propagate
                reward = self.State.giveReward()
                # explicitly assign end state to reward values
                self.state_values[self.State.state] = reward
                print("Game End Reward", reward)
                for s in reversed(self.states):
                    reward = self.state_values[s] + self.lr*(reward - self.state_values[s])
                    self.state_values[s] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                # append trace
                self.states.append(self.State.nxtPosition(action))
                print("current position {} action {}".format(self.State.state, action))
                # by taking the action, it reaches the next state
                self.State = self.takeAction(action)
                # mark is end
                self.State.isEndFunc()
                print("nxt state", self.State.state)
                print("---------------------")
                self.isEnd = self.State.isEnd
    
    def showValues(self):
        for i in range(0, BOARD_ROWS):
            print('----------------------------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                out += str(self.state_values[(i, j)]) + ' | '
            print(out)
        print('----------------------------------')

#### That’s it! These are all we need for a grid world game. We can start and let our agent play the game!

In [103]:

ag = Agent()

ag.play(50)

current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action up
nxt state (2, 1)
---------------------
current position (2, 1) action right
nxt state (2, 2)
---------------------
current position (2, 2) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action down
nxt state (2, 3)
---------------------
current position (2, 3) action up
nxt state (1, 3)
---------------------
Game End Reward -1


In [104]:
ag.showValues()

----------------------------------
| 0.928 | 0.932 | 0.943 | 1.0 | 
----------------------------------
| 0.926 | 0 | 0.841 | -1.0 | 
----------------------------------
| 0.721 | 0.335 | 0.092 | -0.2 | 
----------------------------------


#### This is the estimates of each state after playing 50 rounds of game. As our action is deterministic, we can get best action at each state by following the highest estimate!

##2. Non Deterministic Case
![picture](https://drive.google.com/uc?id=1rs4-aNz9e-wVc-vZn2QGWIojkfXjZmvN)

#### When the agent moving up, it has 0.8 probability moving up and 0.1 probability of going left or right. (non-deterministic)
![picture](https://drive.google.com/uc?id=1cBON9TQMG2iYI1ynODM4cpOJwyVNnyMm)

In [105]:
import numpy as np
BOARD_ROWS = 3
BOARD_COLS = 4
WIN_STATE = (0, 3)
LOSE_STATE = (1, 3)
START = (2, 0)
DETERMINISTIC = False

In [106]:
class State:
    def __init__(self, state=START):
        self.board = np.zeros([BOARD_ROWS, BOARD_COLS])
        self.board[1, 1] = -1
        self.state = state
        self.isEnd = False
        self.determine = DETERMINISTIC
        
    def giveReward(self):
        if self.state == WIN_STATE:
            return 1
        elif self.state == LOSE_STATE:
            return -1
        else:
            return 0
    
    def isEndFunc(self):
        if (self.state == WIN_STATE) or (self.state == LOSE_STATE):
            self.isEnd = True

    def _chooseActionProb(self, action):
        if action == "up":
            return np.random.choice(["up", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "down":
            return np.random.choice(["down", "left", "right"], p=[0.8, 0.1, 0.1])
        if action == "left":
            return np.random.choice(["left", "up", "down"], p=[0.8, 0.1, 0.1])
        if action == "right":
            return np.random.choice(["right", "up", "down"], p=[0.8, 0.1, 0.1])
        
    def nxtPosition(self, action):
        """
        action: up, down, left, right
        -------------
        0 | 1 | 2| 3|
        1 |
        2 |
        return next position on board
        """
        if self.determine:
            if action == "up":
                nxtState = (self.state[0]-1, self.state[1])
            elif action == "down":
                nxtState = (self.state[0]+1, self.state[1])
            elif action == "left":
                nxtState = (self.state[0], self.state[1]-1)
            else:
                nxtState = (self.state[0], self.state[1]+1)
            self.determine = False
        else:
            # non-deterministic
            action = self._chooseActionProb(action)
            self.determine = True
            nxtState = self.nxtPosition(action)
                        
        # if next state is legal
        if (nxtState[0] >= 0) and (nxtState[0] <= 2):
            if (nxtState[1] >= 0) and (nxtState[1] <= 3):
                if nxtState != (1, 1):
                    return nxtState
        return self.state
    
    def showBoard(self):
        self.board[self.state] = 1
        for i in range(0, BOARD_ROWS):
            print('-----------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.board[i, j] == 1:
                    token = '*'
                if self.board[i, j] == -1:
                    token = 'z'
                if self.board[i, j] == 0:
                    token = '0'
                out += token + ' | '
            print(out)
        print('-----------------')

### Action:When the agent exploits the state, it will take the action that maximises the Q value according to current estimation of Q-value.

#### Update Q value: Similar to value iteration, the Q value update is also in a reversed fashion and each update will be conducted at the end of a game.


In [107]:
class Agent:
    
    def __init__(self):
        self.states = []  # record position and action taken at the position
        self.actions = ["up", "down", "left", "right"]
        self.State = State()
        self.isEnd = self.State.isEnd
        self.lr = 0.2
        self.exp_rate = 0.3
        self.decay_gamma = 0.9

        # initial Q values
        self.Q_values = {}
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                self.Q_values[(i, j)] = {}
                for a in self.actions:
                    self.Q_values[(i, j)][a] = 0  # Q value is a dict of dict  
    
    def chooseAction(self):
        # choose action with most expected value
        mx_nxt_reward = float("-inf")
        action = ""
        
        if np.random.uniform(0, 1) <= self.exp_rate:
            action = np.random.choice(self.actions)
        else:
            # greedy action
            for a in self.actions:
                current_position = self.State.state
                nxt_reward = self.Q_values[current_position][a]
                if nxt_reward >= mx_nxt_reward:
                    action = a
                    mx_nxt_reward = nxt_reward
            # print("current pos: {}, greedy aciton: {}".format(self.State.state, action))
        return action
    
    def takeAction(self, action):
        position = self.State.nxtPosition(action)
        # update State
        return State(state=position)     
    
    def reset(self):
        self.states = []
        self.State = State()
        self.isEnd = self.State.isEnd
    
    def play(self, rounds=10):
        i = 0
        while i < rounds:
            # to the end of game back propagate reward
            if self.State.isEnd:
                # back propagate
                reward = self.State.giveReward()
                for a in self.actions:
                    self.Q_values[self.State.state][a] = reward
                print("Game End Reward", reward)
                for s in reversed(self.states):
                    current_q_value = self.Q_values[s[0]][s[1]]
                    reward = current_q_value + self.lr*(self.decay_gamma*reward - current_q_value)
                    self.Q_values[s[0]][s[1]] = round(reward, 3)
                self.reset()
                i += 1
            else:
                action = self.chooseAction()
                # append trace
                self.states.append([(self.State.state), action])
                print("current position {} action {}".format(self.State.state, action))
                # by taking the action, it reaches the next state
                self.State = self.takeAction(action)
                # mark is end
                self.State.isEndFunc()
                print("nxt state", self.State.state)
                print("---------------------")
                self.isEnd = self.State.isEnd

In [108]:
ag = Agent()

ag.Q_values
ag.play(50)

current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action right
nxt state (2, 1)
---------------------
current position (2, 1) action right
nxt state (2, 2)
---------------------
current position (2, 2) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (2, 3)
---------------------
current position (2, 3) action right
nxt state (1, 3)
---------------------
Game End Reward -1
current position (2, 0) action right
nxt state (2, 1)
---------------------
current position (2, 1) action left
nxt state (2, 0)
---------------------
current position (2, 0) action left
nxt state (2, 0)
-----------------

In [109]:
ag.Q_values


{(0, 0): {'down': 0.239, 'left': 0.214, 'right': 0.344, 'up': 0.232},
 (0, 1): {'down': 0.262, 'left': 0.1, 'right': 0.63, 'up': 0.405},
 (0, 2): {'down': 0.119, 'left': 0.428, 'right': 0.78, 'up': 0.598},
 (0, 3): {'down': 1, 'left': 1, 'right': 1, 'up': 1},
 (1, 0): {'down': 0.15, 'left': 0.118, 'right': 0.05, 'up': 0.313},
 (1, 1): {'down': 0, 'left': 0, 'right': 0, 'up': 0},
 (1, 2): {'down': -0.004, 'left': 0.022, 'right': -0.247, 'up': 0.358},
 (1, 3): {'down': -1, 'left': -1, 'right': -1, 'up': -1},
 (2, 0): {'down': 0.09, 'left': 0.102, 'right': 0.083, 'up': 0.242},
 (2, 1): {'down': 0.004, 'left': 0.109, 'right': 0.001, 'up': 0.003},
 (2, 2): {'down': 0.0, 'left': 0.011, 'right': -0.023, 'up': 0},
 (2, 3): {'down': 0, 'left': 0, 'right': -0.163, 'up': 0}}

#### We start at (2, 0) , and the maximum action should be up which values 0.223 and then reaches (1, 0) , from there the best action is still up with value 0.367 , and so on and so forth … Finally we get our policy up -> up -> right -> right -> right . We can see by propagating and updating, our agent is clever enough to generate the best action at each state. And a good thing about Q-learning is that we directly get the best action at each state comparing to basic value iteration.
